In [13]:
from pyspark.sql.functions import * 
from pyspark.sql.types  import * 
from delta.tables import * 


StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 15, Finished, Available, Finished)

In [14]:
DeltaTable.createIfNotExists(spark)\
            .tableName('dimDate_gold')\
            .addColumn('OrderDate',DateType())\
            .addColumn('Day',IntegerType())\
            .addColumn('Month',IntegerType())\
            .addColumn('Year',IntegerType())\
            .addColumn('mmyyyy',StringType())\
            .addColumn('yyyymm',StringType()).execute()


sales_silver_df = spark.read.table('sales.sales_silver')

dimDate_gold_df = sales_silver_df.select('OrderDate').distinct()\
                .select('OrderDate' , \
                year('OrderDate').alias('Year'),\
                month('OrderDate').alias('Month'),\
                dayofmonth('OrderDate').alias('Day'),\
                date_format('OrderDate' , 'yyyyMM').alias('yyyymm'),\
                date_format('OrderDate' , 'MMyyyy').alias('mmyyyy')\
                )

dimDate_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimdate_gold')
update_dict = {"Day": "gold_df.Day", "Month": "gold_df.Month","Year": "gold_df.Year", "mmyyyy": "gold_df.mmyyyy", "yyyymm": "gold_df.yyyymm"}

insert_dict = { "OrderDate" : "gold_df.OrderDate","Day": "gold_df.Day", "Month": "gold_df.Month","Year": "gold_df.Year", "mmyyyy": "gold_df.mmyyyy", "yyyymm": "gold_df.yyyymm"}

dimDate_gold_tbl.alias('gold_tbl')\
            .merge( dimDate_gold_df.alias('gold_df') , \
            "gold_tbl.OrderDate == gold_df.OrderDate")\
            .whenNotMatchedInsert(values =insert_dict).execute()



StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 16, Finished, Available, Finished)

In [15]:
DeltaTable.createIfNotExists(spark)\
            .tableName('dimCustomer_gold')\
            .addColumn('CustomerName',StringType())\
            .addColumn('Email',StringType())\
            .addColumn('First',StringType())\
            .addColumn('Last',StringType())\
            .addColumn('CustomerID',LongType()).execute()

dimCust_gold_df = sales_silver_df.select('CustomerName' , 'Email').distinct()\
                                .select('CustomerName' , 'Email' , \
                                split('CustomerName' , ' ').getItem(0).alias('First'),\
                                split('CustomerName' , ' ').getItem(1).alias('Last'))


dimCust_gold_tbl_tmp =spark.read.table('dimCustomer_gold')
max_customer_id = dimCust_gold_tbl_tmp.select(coalesce(max('CustomerID') , lit(0) )).alias('max_customer_id').first()[0]
dfdimCustomer_gold = dimCust_gold_df.join(dimCust_gold_tbl_tmp ,\
                                (dimCust_gold_df.CustomerName == dimCust_gold_tbl_tmp.CustomerName) & \
                                (dimCust_gold_df.Email == dimCust_gold_tbl_tmp.Email) ,'left_anti')

# display(dfdimCustomer_gold)

dfdimCustomer_gold= dfdimCustomer_gold.withColumn('CustomerId' , monotonically_increasing_id()+max_customer_id+1)

display(dfdimCustomer_gold)




dimCust_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimcustomer_gold')
update_dict = {"Email": "gold_df.Email", "First": "gold_df.First","Last": "gold_df.Last"}

insert_dict = { "CustomerName" : "gold_df.CustomerName","Email": "gold_df.Email", "First": "gold_df.First","Last": "gold_df.Last", "CustomerID": "gold_df.CustomerID"}

dimCust_gold_tbl.alias('gold_tbl')\
            .merge( dfdimCustomer_gold.alias('gold_df') , \
            "gold_tbl.CustomerName == gold_df.CustomerName and gold_tbl.Email == gold_df.Email")\
            .whenNotMatchedInsert(values =insert_dict).execute()
            
            


StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9923f0b8-bbee-442d-8fc9-b2e44df385ad)

In [16]:
DeltaTable.createIfNotExists(spark)\
            .tableName('sales.dimProduct_gold')\
            .addColumn('ItemName',StringType())\
            .addColumn('ItemId',LongType())\
            .addColumn('ItemInfo',StringType()).execute()


dimProd_gold_df = sales_silver_df.select('Item').distinct()\
                                .select( \
                                split('Item' , ',').getItem(0).alias('ItemName'),\
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ).alias('ItemInfo'))




dimProd_gold_tbl_tmp =spark.read.table('dimProduct_gold')
max_Item_Id = dimProd_gold_tbl_tmp.select(coalesce(max('ItemId') , lit(0) )).alias('max_Item_Id').first()[0]
dfdimProd_gold = dimProd_gold_df.join(dimProd_gold_tbl_tmp ,\
                                (dimProd_gold_df.ItemName == dimProd_gold_tbl_tmp.ItemName) & \
                                (dimProd_gold_df.ItemInfo == dimProd_gold_tbl_tmp.ItemInfo) ,'left_anti')

# display(dfdimCustomer_gold)

dfdimProd_gold= dfdimProd_gold.withColumn('ItemId' , monotonically_increasing_id()+max_Item_Id+1)

display(dfdimProd_gold)



dimProd_gold_tbl = DeltaTable.forPath(spark , 'Tables/dimproduct_gold')
update_dict = {}

insert_dict = { "ItemName" : "gold_df.ItemName","ItemInfo": "gold_df.ItemInfo"}

dimProd_gold_tbl.alias('gold_tbl')\
            .merge( dfdimProd_gold.alias('gold_df') , \
            "gold_tbl.ItemName == gold_df.ItemName and gold_tbl.ItemInfo == gold_df.ItemInfo")\
            .whenNotMatchedInsert(values =insert_dict).execute()


StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0b87cc6-ffeb-452f-a6da-c612dcb25c9b)

In [7]:
DeltaTable.createIfNotExists(spark)\
            .tableName('sales.factSales_gold')\
            .addColumn('CustomerID',LongType())\
            .addColumn('ItemId',LongType())\
            .addColumn('OrderDate',DateType())\
            .addColumn('Quantity',IntegerType())\
            .addColumn('UniPrice',LongType())\
            .addColumn('Tax',IntegerType()).execute()


StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 9, Finished, Available, Finished)

In [ ]:
df_dim_cust_tmp = spark.read.format('sales.dimcustomer_gold')
df_dim_prod_tmp = spark.read.format('sales.dimproduct_gold')


sales_silver_df = sales_silver_df.withColumn(\
                                'ItemName',split('Item' , ',').getItem(0))\
                                .withColumn('ItemInfo' , \
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ) )


display(sales_silver_df.head(10))



In [ ]:



# Create Sales_gold dataframe

dffactSales_gold = df.alias("df1").join(dfdimCustomer_temp.alias("df2"),(df.CustomerName == dfdimCustomer_temp.CustomerName) & (df.Email == dfdimCustomer_temp.Email), "left") \
       .join(dfdimProduct_temp.alias("df3"),(df.ItemName == dfdimProduct_temp.ItemName) & (df.ItemInfo == dfdimProduct_temp.ItemInfo), "left") \
   .select(col("df2.CustomerID") \
       , col("df3.ItemID") \
       , col("df1.OrderDate") \
       , col("df1.Quantity") \
       , col("df1.UnitPrice") \
       , col("df1.Tax") \
   ).orderBy(col("df1.OrderDate"), col("df2.CustomerID"), col("df3.ItemID"))

# Display the first 10 rows of the dataframe to preview your data

display(dffactSales_gold.head(10))


dimSales_gold_tbl = DeltaTable.forPath(spark , 'Tables/factsales_gold')
update_dict = {}

insert_dict = { "CustomerID" : "gold_df.CustomerID","ItemId": "gold_df.ItemId" , "OrderDate" : "gold_df.OrderDate","Quantity":"gold_df.Quantity" , \
                "UnitPrice":"gold_df.UnitPrice" , "Tax" : "gold_df.Tax" }

dimProd_gold_tbl.alias('gold_tbl')\
            .merge( dfdimProd_gold.alias('gold_df') , \
            "gold_tbl.CustomerID == gold_df.CustomerID and gold_tbl.ItemId == gold_df.ItemId\
            and gold_tbl.OrderDate == gold_df.OrderDate") \
            .whenNotMatchedInsert(values =insert_dict).execute()





            


In [32]:
df_dim_cust_tmp = spark.read.table('sales.dimcustomer_gold')
display(df_dim_cust_tmp.head(10))

StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cc2e6af5-e241-4b3e-8f9a-9cfba6e80a31)

In [31]:
#########################################

df_dim_cust_tmp = spark.read.table('sales.dimcustomer_gold')
df_dim_prod_tmp = spark.read.table('sales.dimproduct_gold')


sales_silver_df = sales_silver_df.withColumn(\
                                'ItemName',split('Item' , ',').getItem(0))\
                                .withColumn('ItemInfo' , \
                                when( ( (split('Item' , ',').getItem(1).isNull()) | (split('Item' , ' ').getItem(1)=='') ) , 'NULL' )\
                                .otherwise(split('Item' , ',').getItem(1) ) )


display(df_dim_cust_tmp.head(10))

factSale_gold_df = sales_silver_df.join(df_dim_cust_tmp , (sales_silver_df.CustomerName == df_dim_cust_tmp.CustomerName) & (sales_silver_df.Email == df_dim_cust_tmp.Email) , "left")\
                                  .join(df_dim_prod_tmp , (sales_silver_df.ItemName == df_dim_prod_tmp.ItemName) & (sales_silver_df.ItemInfo == df_dim_prod_tmp.ItemInfo) , "left")\
                                  .select(\
                                  col('df_dim_prod_tmp.ItemId'),\
                                  col('sales_silver_df.OrderDate'),\
                                  col('sales_silver_df.Tax'),\
                                  col('sales_silver_dfUnitPrice'),\
                                  col('sales_silver_df.Quantity'))

                                  

display(factSale_gold_df.head(10))
#########################################

StatementMeta(, 4e68bd5a-60e3-483d-b72d-b9ffe6a55721, 35, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9a1a5695-b1a8-41ee-a998-74185ce1c37f)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `df_dim_prod_tmp`.`ItemId` cannot be resolved. Did you mean one of the following? [`ItemName`, `ItemInfo`, `spark_catalog`.`sales`.`dimproduct_gold`.`ItemId`, `spark_catalog`.`sales`.`sales_silver`.`Item`, `spark_catalog`.`sales`.`dimproduct_gold`.`ItemInfo`].;
'Project ['df_dim_prod_tmp.ItemId, 'sales_silver_df.OrderDate, 'sales_silver_df.Tax, 'sales_silver_dfUnitPrice, 'sales_silver_df.Quantity]
+- Join LeftOuter, ((ItemName#9496 = ItemName#9490) AND (ItemInfo#9512 = ItemInfo#9492))
   :- Join LeftOuter, ((CustomerName#5365 = CustomerName#9480) AND (Email#5366 = Email#9481))
   :  :- Project [SalesOrderNumber#5362, SalesOrderLineNumber#5363, OrderDate#5364, CustomerName#5365, Email#5366, Item#5367, Quantity#5368, UnitPrice#5369, Tax#5370, FileName#5371, IsFlagged#5372, CreatedTS#5373, ModifiedTS#5374, ItemName#9496, CASE WHEN (isnull(split(Item#5367, ,, -1)[1]) OR (split(Item#5367,  , -1)[